In [2]:
import numpy as np

## Main problems in making a simulation:
1. In solving CS model with solve_ivp - the main problem is write fast rhs function.
2. Making an animation out of data.

### 0. Before you start to write your RHS functions.
(in this case) it will be better to start with making functions which generate random data inputs
(or at least generate one) to test pieces of your code on every step while you are writing.

In [16]:
from numpy.random import default_rng, SeedSequence

### initial condition functions ###
def set_random_y0(N, seed, low=0, high=1, velo_st_dev=4):
    """Produces an initial CS_matrix"""
    rng = default_rng(SeedSequence(seed))
    
    x_s = rng.uniform(low=low, high=high, size=N) # x coordinates of intial position
    y_s = rng.uniform(low=low, high=high, size=N) # y coordinates of intial position

    vx_s = velo_st_dev*rng.standard_normal(N) # x coordinates of intial velocity
    vy_s = velo_st_dev*rng.standard_normal(N) # y coordinates of intial velocity

    y_0 = np.c_[x_s, y_s, vx_s, vy_s] #returns a CS_matrix - initial condition
    return y_0
set_random_y0(5,1234)

array([[ 0.97669977,  0.11809123, -2.04977484, -8.63655605],
       [ 0.38019574,  0.24176629,  5.29503583,  1.7389358 ],
       [ 0.92324623,  0.31853393, -3.44112077,  6.93315728],
       [ 0.26169242,  0.96407925,  2.0779728 ,  2.08053662],
       [ 0.31909706,  0.2636498 , -5.06057487, -4.00866318]])

### 1. CS right hand side
We are going to write rhs for model in which every bird "see" every other bird, hence the equation for the behaviour of $i^{\text{th}}$ bird is:
\begin{equation}
\begin{split}
\dot{x_i} & = v_i \\
\dot{v_i} & = \sum_j \eta\!\left( \text{dist}(x_j,x_i)\right)\cdot\left(v_j-v_i\right)
\end{split}
\end{equation}
where $$\eta(s) = \frac{K}{(\sigma^2+s)^\beta}.$$

%%latex
Notice that we are going to solve system of $N \times 4$ functions.\
We organize data into $N \times 4$ matrix, every row is a bird which is described by 4 numbers: 
first two - its position (call it temporarily $x_i$), last two - its velocity (call it temporarily $v_i$). This matrix is an input for the right hand side function.\
solve_ivp solves dy / dt = f(t, y) thus we can imagine we have an equation that looks like:
$$
%\overset{\bullet}{
\begin{pmatrix}
 x_1 & v_1 \\
 x_2 & v_2 \\
 \vdots & \vdots \\
 x_N & v_N
\end{pmatrix}'
%}
= 
\begin{pmatrix}
 v_1 & \sum\eta\!\left( \text{dist}(x_j,x_1)\right)\cdot\left(v_j-v_1\right) \\
 v_2 & \sum\eta\!\left( \text{dist}(x_j,x_2)\right)\cdot\left(v_j-v_2\right) \\
 \vdots & \vdots \\
 v_N & \sum\eta\!\left( \text{dist}(x_j,x_N)\right)\cdot\left(v_j-v_N\right)
\end{pmatrix}
$$
and we have to construct a function which for the matrix $y = [x\:v]$ constructs what you see on the right of the equation sign above.

In [50]:
""" Lets define some constants and communication weight"""
N = 5
def comm_weight(s):
    """Communication weight in Cucker-Smale model"""
    return np.float_power(s, -0.8, out=np.zeros_like(s), where=s!=0)

#### Why this:
```python
,where=s!=0
```
The strategy:
1. Split function rhs into two: one returns derivatives of position, the second returns the derivaties of velocities,
2. In the second using numpy arrays and numpy operations we construct the expression above while the first function(derivative of position) returns second two rows of the input.

In [13]:
from typing import Callable

def rhs_func(t: float, y: np.ndarray, mass: np.ndarray, comm_weight: Callable)-> np.ndarray:
    """ The splitting of dx and dv is done here. 
            t           : independent variable in the ODE
            y           : CS_data_point of the shape (N*4,)
            mass        : array of "masses" we can set that some birds are more important than other
            comm_weight : callable 'communication weight' funtion
        """
    N = len(m)
    y = y.reshape((N,4))    # this is for technical reasons = solve_ivp was not working for me with the .shape =(N, 4) so i'm using .shape=(N*4,)
    x_derivatives = rhs_d_pos(y)
    y_derivatives = rhs_d_vel(y, mass,comm_weight)
    rhs = x_derivatives + y_derivatives
    return rhs.ravel()

We have to define two funtions: rhs_d_pos and rhs_d_vel.\
Another benefit of those functions is that we are back at the shape (n,4).

In [30]:
""" return of rhs_d_pos will have the shape as input and compatible with last line rhs_func """
def rhs_d_pos(y):
    u = y.copy()
    u[:,0], u[:,1] = 0, 0    # zeroing the position
    return u[:,[2,3,0,1]]    # velocities go to the front

# print(set_random_y0(5,1234)) 
# rhs_d_pos(set_random_y0(5,1234)) # check

In [35]:
""" Now the harder part: rhs_d_vel """
def rhs_d_vel(y, mass=0, comm_weight=0):
    vel = y.copy()
    vel[:,0], vel[:,1] = 0, 0   # zeroing the position
    pos = y.copy()
    pos[:,2], pos[:,3] = 0, 0   # zeroing the velocity
    result = np.zeros(y.shape)  # place for the result
    
    # creating list of matricies of differences .shape=(N,N,4)
    
    
    return result
    
# print(set_random_y0(5,1234)) 
# rhs_d_vel(set_random_y0(5,1234)) # check

[[ 0.97669977  0.11809123 -2.04977484 -8.63655605]
 [ 0.38019574  0.24176629  5.29503583  1.7389358 ]
 [ 0.92324623  0.31853393 -3.44112077  6.93315728]
 [ 0.26169242  0.96407925  2.0779728   2.08053662]
 [ 0.31909706  0.2636498  -5.06057487 -4.00866318]]


array([[0.97669977, 0.11809123, 0.        , 0.        ],
       [0.38019574, 0.24176629, 0.        , 0.        ],
       [0.92324623, 0.31853393, 0.        , 0.        ],
       [0.26169242, 0.96407925, 0.        , 0.        ],
       [0.31909706, 0.2636498 , 0.        , 0.        ]])

In [47]:
# creating list of matricies of differences .shape=(N,N,4)
u = set_random_y0(5,1234)
u[:,0], u[:,1] = 0, 0    # zeroing the position

# inspect the following two lines:
# np.stack([u]*N,axis=0)
# np.stack([u]*N,axis=1) 
vel_diff = np.stack([u]*N,axis=0) - np.stack([u]*N,axis=1)
vel_diff.shape

(5, 5, 4)